In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
result = requests.get("https://jobs.lever.co/aquicore")
c = result.content
soup = bs(c)

Let's see if we can grab the titles from the *h5* tags. 

In [3]:
titles = soup.find_all('h5')
titles

[<h5>Account Executive</h5>,
 <h5>Solutions Architect</h5>,
 <h5>Data Scientist</h5>,
 <h5>Embedded Software Engineer</h5>,
 <h5>Full Stack Software Engineer</h5>,
 <h5>Product Manager</h5>,
 <h5>Quality Engineer</h5>,
 <h5>Technical Support Engineer</h5>]

Cool - that worked.

Now let's give them numbers.

In [4]:
l_titles = [title.text.strip() for title in titles]
l_titles

['Account Executive',
 'Solutions Architect',
 'Data Scientist',
 'Embedded Software Engineer',
 'Full Stack Software Engineer',
 'Product Manager',
 'Quality Engineer',
 'Technical Support Engineer']

Alright. Let's read that into a DataFrame. 

In [5]:
df = pd.DataFrame(l_titles)
df.columns=['Position']
df['Company'] = 'Aquicore'
df

,Position,Company
0,Account Executive,Aquicore
1,Solutions Architect,Aquicore
2,Data Scientist,Aquicore
3,Embedded Software Engineer,Aquicore
4,Full Stack Software Engineer,Aquicore
5,Product Manager,Aquicore
6,Quality Engineer,Aquicore
7,Technical Support Engineer,Aquicore


Now let's read in the location as a new column

In [ ]:
df['Location']=